# Dask Delayed

Материалы:
* Макрушин С.В. Лекция 13: Dask Delayed
* https://docs.dask.org/en/latest/delayed.html
* JESSE C. DANIEL. Data Science with Python and Dask.


## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

## Лабораторная работа 14

1. Напишите функцию, которая считывает файл формата xml из архива `reviewers_full.zip` и по данным этого файла формирует список словарей, содержащих следующие ключи: `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



In [28]:
from bs4 import BeautifulSoup
import dask
from dask import delayed
from dask import bag

In [3]:
with open('reviewers_full/reviewers_full_0.xml') as f:
    data = BeautifulSoup(f, 'xml')

In [7]:
raw = data.find_all('user')
new_data = []
for d in raw:
    new_dict = {
        'username': d.username.text if d.username else None,
        'name': d.find('name').text if d.find('name') else None,
        'sex': d.sex.text if d.sex else None,
        'country': d.country.text if d.county else None,
        'mail': d.mail.text if d.mail else None,
        'registered': d.registered.text if d.registered else None,
        'birthdate': d.birthdate.text if d.birthdate else None,
        'name_prefix': d.attrs.get('prefix', None),
        'country_code': d.country.attrs.get('code', None) if d.country else None
    }
    new_data.append(new_dict)
new_data   

[{'username': 'gabrielacalhoun',
  'name': None,
  'sex': 'F',
  'country': None,
  'mail': None,
  'registered': None,
  'birthdate': '1988-01-25',
  'name_prefix': 'Mrs.',
  'country_code': None},
 {'username': 'qbaxter',
  'name': None,
  'sex': None,
  'country': None,
  'mail': 'qware@gmail.com',
  'registered': None,
  'birthdate': '1985-01-19',
  'name_prefix': None,
  'country_code': 'NO'},
 {'username': 'crosschristopher',
  'name': 'Dana Moore',
  'sex': None,
  'country': None,
  'mail': 'stephaniestrong@yahoo.com',
  'registered': '2018-11-21',
  'birthdate': '1955-07-03',
  'name_prefix': None,
  'country_code': None},
 {'username': 'karen27',
  'name': 'Jennifer Horne',
  'sex': None,
  'country': None,
  'mail': 'wjarvis@yahoo.com',
  'registered': '2013-11-20',
  'birthdate': '2007-04-30',
  'name_prefix': None,
  'country_code': 'CU'},
 {'username': 'gambledanielle',
  'name': 'Henry Harris',
  'sex': None,
  'country': None,
  'mail': None,
  'registered': '2011-04-08

2. Измерьте время выполнения функции из задания 1 на всех файлах из архива. Ускорьте время выполнения, используя `dask.delayed`.

In [20]:
def transfer_data(raw):
    new_data = []
    for d in raw:
        i = {
            'username': d.username.text if d.username else None,
            'name': d.find('name').text if d.find('name') else None,
            'sex': d.sex.text if d.sex else None,
            'country': d.country.text if d.county else None,
            'mail': d.mail.text if d.mail else None,
            'registered': d.registered.text if d.registered else None,
            'birthdate': d.birthdate.text if d.birthdate else None,
            'name_prefix': d.attrs.get('prefix', None),
            'country_code': d.country.attrs.get('code', None) if d.country else None
        }
        new_data.append(i)
    return new_data
    

In [22]:
d = delayed(transfer_data)(raw)
d = d.compute()
d

[{'username': 'gabrielacalhoun',
  'name': None,
  'sex': 'F',
  'country': None,
  'mail': None,
  'registered': None,
  'birthdate': '1988-01-25',
  'name_prefix': 'Mrs.',
  'country_code': None},
 {'username': 'qbaxter',
  'name': None,
  'sex': None,
  'country': None,
  'mail': 'qware@gmail.com',
  'registered': None,
  'birthdate': '1985-01-19',
  'name_prefix': None,
  'country_code': 'NO'},
 {'username': 'crosschristopher',
  'name': 'Dana Moore',
  'sex': None,
  'country': None,
  'mail': 'stephaniestrong@yahoo.com',
  'registered': '2018-11-21',
  'birthdate': '1955-07-03',
  'name_prefix': None,
  'country_code': None},
 {'username': 'karen27',
  'name': 'Jennifer Horne',
  'sex': None,
  'country': None,
  'mail': 'wjarvis@yahoo.com',
  'registered': '2013-11-20',
  'birthdate': '2007-04-30',
  'name_prefix': None,
  'country_code': 'CU'},
 {'username': 'gambledanielle',
  'name': 'Henry Harris',
  'sex': None,
  'country': None,
  'mail': None,
  'registered': '2011-04-08

3. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

In [33]:
reviewers = []
for i in range(5):
    with open(f'reviewers_full/reviewers_full_{i}.xml') as f:
        data = BeautifulSoup(f, 'xml')
        raw = data.find_all('user')
        d = delayed(transfer_data)(raw)
        reviewers.append(d)

In [34]:
bag_d = bag.from_delayed(reviewers)

In [35]:
rev_data = dask.compute(*reviewers)

In [36]:
rev_data = 

([{'username': 'gabrielacalhoun',
   'name': None,
   'sex': 'F',
   'country': None,
   'mail': None,
   'registered': None,
   'birthdate': '1988-01-25',
   'name_prefix': 'Mrs.',
   'country_code': None},
  {'username': 'qbaxter',
   'name': None,
   'sex': None,
   'country': None,
   'mail': 'qware@gmail.com',
   'registered': None,
   'birthdate': '1985-01-19',
   'name_prefix': None,
   'country_code': 'NO'},
  {'username': 'crosschristopher',
   'name': 'Dana Moore',
   'sex': None,
   'country': None,
   'mail': 'stephaniestrong@yahoo.com',
   'registered': '2018-11-21',
   'birthdate': '1955-07-03',
   'name_prefix': None,
   'country_code': None},
  {'username': 'karen27',
   'name': 'Jennifer Horne',
   'sex': None,
   'country': None,
   'mail': 'wjarvis@yahoo.com',
   'registered': '2013-11-20',
   'birthdate': '2007-04-30',
   'name_prefix': None,
   'country_code': 'CU'},
  {'username': 'gambledanielle',
   'name': 'Henry Harris',
   'sex': None,
   'country': None,
   

4. Из `dask.bag`, полученного в задании 3, создайте `dask.dataframe` при помощи метода `bag.to_dataframe`. Укажите столбец `id` в качестве индекса.

5. Назовем отзыв негативным, если оценка равна 0, 1 или 2. Загрузите данные о негативных отзывах из файлов архива `reviews_full` (__ЛР12__) в виде `dask.DataFrame`. Посчитайте количество отзывов с группировкой по пользователю, оставившему отзыв. Объедините результат с таблицей, полученной в задаче 4.